# Week 6 worked example 2
## Linearized supersonic flow

Dr. Daniel Duke<br>
Laboratory for Turbulence Research in Aerospace & Combustion (LTRAC)<br>
Department of Mechanical & Aerospace Engineering<br>
Monash University, Australia

Calculate the lift and wave-drag coefficients for flow over a flat plate and compare to the results of ideal shock-expansion theory.

Linearized theory for flat plates:
$$
\begin{align}
    c_l = \frac{4 \alpha}{\sqrt{M_\infty^2 - 1}}\\
    c_d = \frac{4 \alpha^2}{\sqrt{M_\infty^2 - 1}}\\
\end{align}
$$

Ideal shock expansion theory - there will be an expansion fan on the upper surface leading edge and an  oblique shock on the lower surface leading edge. The solution procedure is:

Find the presssure downstream of the oblique shock:
$$
\begin{align}
    \tan \theta = \tan \alpha = 2 \cot \beta \frac{ M_1^2 \sin^2 \beta - 1} {M_1^2 \left( \gamma + \cos 2 \beta \right) + 2 }\\
    \\
    \frac{p_3}{p_\infty} = \frac{2 \gamma M_\infty^2 \sin^2 \beta - (\gamma-1)}{\gamma + 1}\\
\end{align}
$$

Find the pressure downstream of the expansion fan:
$$
\begin{align}
    \alpha = \nu(M_2) - \nu(M_\infty)\\
    \\
    p_0 = p_\infty \left( 1 + \frac{\gamma - 1}{2} M_\infty^2 \right)^{\frac{\gamma}{\gamma-1}}\\
    \\
    p_2 = p_0 \left( 1 + \frac{\gamma - 1}{2} M_2^2 \right)^{\frac{-\gamma}{\gamma-1}}\\
\end{align}
$$

Solve for the lift and drag coefficients
$$
\begin{align}
    c_l = \frac{2 (p_3 - p_2)}{\gamma p_1 M_\infty^2}\\
    \\
    c_d = c_l \frac{\sin \alpha}{\cos \alpha}
\end{align}
$$





In [3]:
# Import python libraries
import numpy as np
import scipy.optimize as sopt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
%matplotlib notebook

In [11]:
# Define constants
gamma = 1.4 # air

# Define variables
#Minf_slider=widgets.FloatSlider(min=1.2,max=5.0,step=0.1,value=2.0)
Minf = np.arange(1.2,5.0,0.1)
AoA_slider=widgets.FloatSlider(min=0,max=10,step=0.1,value=4.0)

In [26]:
# Functions for the above equations. Angles in degrees.
def cot(theta): return 1.0/np.tan(theta)
    
def oblique_beta(theta, M1, gamma):

    # this function has a root for correct pair of beta and theta
    def zerofun(beta,theta,M1,gamma):
        return (2*cot(beta)*(M1**2 * np.sin(beta)**2 - 1)/(M1**2 * (gamma + np.cos(2*beta))+2)) - np.tan(theta)
    
    #print(zerofun(np.pi*.5,theta,M1,gamma))
    root = sopt.brentq(zerofun, -np.pi, np.pi, args=(theta*np.pi/180.,M1,gamma))
    return root * 180./np.pi

def oblique_pressure_ratio(beta, M1, gamma):
    return (2*gamma*(M1**2)*np.sin(beta)**2 - (gamma-1))/(gamma+1)

def stagn_pressure_ratio(gamma,M):
    return (1 + 0.5*(gamma-1)*M**2)**(gamma/(gamma-1))

# AoA alpha in degrees
def ideal_shock_cl_cd(pl_pinf,pu_pinf,M,gamma,alpha):
    cl = (2/(gamma*M**2)) * ( pl_pinf - pu_pinf )
    cd = cl * np.sin(alpha *np.pi/180. ) / np.cos(alpha*np.pi/180.)
    return cl,cd

# Return nu in degrees
def prandtl_meyer(M,gamma):
    A = np.sqrt((gamma+1.)/(gamma-1.))
    B = np.sqrt((M**2 - 1.)*(gamma-1.)/(gamma+1.))
    C = np.sqrt(M**2 - 1.)
    nu = A * np.arctan(B) - np.arctan(C)
    return nu * 180. / np.pi

In [24]:
# Routine to solve for cl, cd for a range of mach numbers at fixed angle of attach
def run_calcs(M1_range, alpha, gamma):
    
    cl_all = []
    cd_all = []
    
    for M1 in M1_range:
    
        beta = oblique_beta(alpha, M1, gamma)
        pl_pinf = oblique_pressure_ratio(beta, M1, gamma)

        nu1 = prandtl_meyer(M1, gamma)
        nu2 = alpha + nu1
        M2 = sopt.brentq(lambda M: prandtl_meyer(M,gamma)-nu2, 1.0, 5.0)
        p0_pinf = stagn_pressure_ratio(gamma,M1)
        p0_p2 = stagn_pressure_ratio(gamma,M2)
        pu_pinf = p0_pinf / p0_p2

        cl, cd = ideal_shock_cl_cd(pl_pinf,pu_pinf,M1,gamma,alpha)
        
        cl_all.append(cl)
        cd_all.append(cd)
        
    return np.array(cl_all), np.array(cd_all)
        

In [27]:
# Main solver

# set up 
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title("$\alpha = %.2f^{\circ}$" % AoA_slider.value)
plt.grid(alpha=.2)
plt.xlabel("$M_\infty$")

# calculate cl,cd
cl,cd = run_calcs(Minf, AoA_slider.value, gamma)

# now plot
g1,=ax.plot(Minf, cl)
g2,=ax.plot(Minf, cd)

def update_graph(alpha):

    # recalculate cl,cd
    cl,cd = run_calcs(Minf, AoA_slider.value, gamma)
    
    #g1.set_xdata(Minf)
    g1.set_ydata(cl)
    g2.set_xdata(Minf)
    g2.set_ydata(cd)
    
    ax.set_title("$\alpha = %.2f^{\circ}$" % AoA_slider.value)
    fig.canvas.draw_idle()
    
    return
    

interact(update_graph, alpha=AoA_slider);

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


ValueError: f(a) and f(b) must have different signs